## Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

<hr style="border:2px solid black"> </hr>

## Classifier class and functions

Fisher linear classifier class

In [2]:
class FisherLinearClassifier:
    def __init__(self):
        self.w_tilda = []
        self.labels = []
    
    def train(self, X_train, Y_train, labels):
        number_of_features = X_train.shape[1]
        X_class1 = []
        X_class2 = []

        for i in range (0, len(X_train)):
            if Y_train[i] == labels[0]:
                X_class1.append(X_train[i])
            else:
                X_class2.append(X_train[i])

        X_class1 = np.array(X_class1)
        X_class2 = np.array(X_class2)
        mean_class1 = np.mean(X_class1, axis=0)
        mean_class2 = np.mean(X_class2, axis=0)

        Sw_class1 = np.zeros((number_of_features, number_of_features))
        Sw_class2 = np.zeros((number_of_features, number_of_features))
        
        for i in range(0, X_class1.shape[0]):
            Sw_class1 += ((X_class1[i] - mean_class1).reshape(-1, 1) @ (X_class1[i] - mean_class1).reshape(-1, 1).T)
        for i in range(0, X_class2.shape[0]):
            Sw_class2 += ((X_class2[i] - mean_class2).reshape(-1, 1) @ (X_class2[i] - mean_class2).reshape(-1, 1).T)
        Sw_inv = np.linalg.pinv(Sw_class1 + Sw_class2)

        w = Sw_inv @ (mean_class1 - mean_class2)
        w_node = -0.5 * (w.T @ (mean_class1 + mean_class2))
        self.labels = labels
        self.w_tilda = np.append(w, w_node).reshape(-1, 1)
    
    def predict_value(self, feature_vector):
        result = self.w_tilda.T @ np.append(feature_vector, 1).reshape(-1, 1)
        return result[0][0]
    
    def predict_label(self, feature_vector):
        y = self.predict(feature_vector)
        if (y > 0):
            return labels[0]
        return labels[1]

Function to get the confusion matrix

In [3]:
def get_confusion_matrix(Y_test, Y_predict, labels):
    number_of_classes = len(labels)
    confusion_matrix = np.zeros((number_of_classes, number_of_classes))
    for i in range(len(Y_test)):
        row = labels.index(Y_test[i])
        column = labels.index(Y_predict[i])
        confusion_matrix[row][column] += 1
    return confusion_matrix

Function to convert image to grayscale feature vector

In [4]:
def convert_to_grayscale_vector(image):
    feature_vector = []
    for j in range(image.shape[0]):
        for k in range(image.shape[1]):
            r = image[j][k][0]
            g = image[j][k][1]
            b = image[j][k][2]
            grayscale_value = 0.3*r + 0.59*g + 0.11*b
            feature_vector.append(grayscale_value)
    return feature_vector

<hr style="border:2px solid black"> </hr>

## Getting the images and their labels

Read training and testing images

In [5]:
airplane_train_images = np.array([plt.imread("Data/Train/airplane/"+str(i)+".jpg") for i in range(0, 5000)])
automobile_train_images = np.array([plt.imread("Data/Train/automobile/"+str(i)+".jpg") for i in range(0, 5000)])
frog_train_images = np.array([plt.imread("Data/Train/frog/"+str(i)+".jpg") for i in range(0, 5000)])

airplane_test_images = np.array([plt.imread("Data/Test/airplane/"+str(i)+".jpg") for i in range(0, 1000)])
automobile_test_images = np.array([plt.imread("Data/Test/automobile/"+str(i)+".jpg") for i in range(0, 1000)])
frog_test_images = np.array([plt.imread("Data/Test/frog/"+str(i)+".jpg") for i in range(0, 1000)])

Getting separate labels for each class train data

In [6]:
Y_airplane_train = ["airplane" for i in range(airplane_train_images.shape[0])]
Y_airplane_train += ["not_airplane" for i in range(automobile_train_images.shape[0]+frog_train_images.shape[0])]
airplane_labels = ["airplane", "not_airplane"]

Y_automobile_train = ["not_automobile" for i in range(airplane_train_images.shape[0])]
Y_automobile_train += ["automobile" for i in range(automobile_train_images.shape[0])]
Y_automobile_train += ["not_automobile" for i in range(frog_train_images.shape[0])]
automobile_labels = ["automobile", "not_automobile"]

Y_frog_train = ["not_frog" for i in range(airplane_train_images.shape[0]+automobile_train_images.shape[0])]
Y_frog_train += ["frog" for i in range(frog_train_images.shape[0])]
frog_labels = ["frog", "not_frog"]

Getting labels for all the classes train and test data

In [7]:
Y_train = ["airplane" for i in range(airplane_train_images.shape[0])]
Y_train += ["automobile" for i in range(automobile_train_images.shape[0])]
Y_train += ["frog" for i in range(frog_train_images.shape[0])]

Y_test = ["airplane" for i in range(airplane_test_images.shape[0])]
Y_test += ["automobile" for i in range(automobile_test_images.shape[0])]
Y_test += ["frog" for i in range(frog_test_images.shape[0])]

labels = ["airplane", "automobile", "frog"]

<hr style="border:2px solid black"> </hr>

## Testing RGB

Transforming the images into feature vectors and normalizing them

In [8]:
X_airplane_train = np.array([airplane_train_images[i].reshape(-1) for i in range(0, airplane_train_images.shape[0])])
X_automobile_train = np.array([automobile_train_images[i].reshape(-1) for i in range(0, automobile_train_images.shape[0])])
X_frog_train = np.array([frog_train_images[i].reshape(-1) for i in range(0, frog_train_images.shape[0])])

X_airplane_test = np.array([airplane_test_images[i].reshape(-1) for i in range(0, airplane_test_images.shape[0])])
X_automobile_test = np.array([automobile_test_images[i].reshape(-1) for i in range(0, automobile_test_images.shape[0])])
X_frog_test = np.array([frog_test_images[i].reshape(-1) for i in range(0, frog_test_images.shape[0])])

X_train = np.concatenate((X_airplane_train, X_automobile_train, X_frog_train)) / 255.0
X_test = np.concatenate((X_airplane_test, X_automobile_test, X_frog_test)) / 255.0

Construct Fischer's Linear Discriminant classifier for each of the 3 classes

In [9]:
airplane_classifier = FisherLinearClassifier()
airplane_classifier.train(X_train, Y_airplane_train, airplane_labels)

In [ ]:
automobile_classifier = FisherLinearClassifier()
automobile_classifier.train(X_train, Y_automobile_train, automobile_labels)

In [ ]:
frog_classifier = FisherLinearClassifier()
frog_classifier.train(X_train, Y_frog_train, frog_labels)

Predict the label of the test images using the 3 classifers

In [ ]:
Y_predict = []
for i in range(X_test.shape[0]):
    y1 = airplane_classifier.predict_value(X_test[i])
    y2 = automobile_classifier.predict_value(X_test[i])
    y3 = frog_classifier.predict_value(X_test[i])
    if y1 >= y2 and y1>= y3:
        Y_predict.append("airplane")
    elif y2 >= y1 and y2>= y3:
        Y_predict.append("automobile")
    else:
        Y_predict.append("frog")

Calculating the confusion matrix from the predicted labels

In [ ]:
confusion_matrix = get_confusion_matrix(Y_test, Y_predict, labels)

Plotting the results

In [ ]:
accuracy = np.diag(confusion_matrix).sum()/X_test.shape[0]
plt.rc("figure", figsize=[5,5])
plt.matshow(confusion_matrix,cmap="Blues")
plt.title("FLD+RGB = {0:0.3f}".format(accuracy))
for i in range(0,confusion_matrix.shape[0]):
    for j in range(0,confusion_matrix.shape[1]):
        plt.annotate(confusion_matrix[i,j],(j,i))
plt.savefig("figure_rgb")

<hr style="border:2px solid black"> </hr>

## Testing Grayscale

Transforming the images into feature vectors and normalizing them

In [ ]:
X_airplane_train = np.array([convert_to_grayscale_vector(airplane_train_images[i]) for i in range(0, airplane_train_images.shape[0])])
X_automobile_train = np.array([convert_to_grayscale_vector(automobile_train_images[i]) for i in range(0, automobile_train_images.shape[0])])
X_frog_train = np.array([convert_to_grayscale_vector(frog_train_images[i]) for i in range(0, frog_train_images.shape[0])])

X_airplane_test = np.array([convert_to_grayscale_vector(airplane_test_images[i]) for i in range(0, airplane_test_images.shape[0])])
X_automobile_test = np.array([convert_to_grayscale_vector(automobile_test_images[i]) for i in range(0, automobile_test_images.shape[0])])
X_frog_test = np.array([convert_to_grayscale_vector(frog_test_images[i]) for i in range(0, frog_test_images.shape[0])])

X_train = np.concatenate((X_airplane_train, X_automobile_train, X_frog_train)) / 255.0
X_test = np.concatenate((X_airplane_test, X_automobile_test, X_frog_test)) / 255.0

Construct Fischer's Linear Discriminant classifier for each of the 3 classes

In [ ]:
airplane_classifier = FisherLinearClassifier()
airplane_classifier.train(X_train, Y_airplane_train, airplane_labels)

In [ ]:
automobile_classifier = FisherLinearClassifier()
automobile_classifier.train(X_train, Y_automobile_train, automobile_labels)

In [ ]:
frog_classifier = FisherLinearClassifier()
frog_classifier.train(X_train, Y_frog_train, frog_labels)

Predict the label of the test images using the 3 classifers

In [ ]:
Y_predict = []
for i in range(X_test.shape[0]):
    y1 = airplane_classifier.predict_value(X_test[i])
    y2 = automobile_classifier.predict_value(X_test[i])
    y3 = frog_classifier.predict_value(X_test[i])
    if y1 >= y2 and y1>= y3:
        Y_predict.append("airplane")
    elif y2 >= y1 and y2>= y3:
        Y_predict.append("automobile")
    else:
        Y_predict.append("frog")

Calculating the confusion matrix from the predicted labels

In [ ]:
confusion_matrix = get_confusion_matrix(Y_test, Y_predict, labels)

Plotting the results

In [ ]:
accuracy = np.diag(confusion_matrix).sum()/X_test.shape[0]
plt.rc("figure", figsize=[5,5])
plt.matshow(confusion_matrix,cmap="Blues")
plt.title("FLD+GRAY = {0:0.3f}".format(accuracy))
for i in range(0,confusion_matrix.shape[0]):
    for j in range(0,confusion_matrix.shape[1]):
        plt.annotate(confusion_matrix[i,j],(j,i))
plt.savefig("figure_grayscale")

<hr style="border:2px solid black"> </hr>

## Comments

The accuracy decreased when the images were converted to grayscale than when the RGB representation was used. This is because of the details that are lost when we convert the image to grayscale, due to the loss of colour information from the image. The frog class had the biggest change in accuracy dropping from 78.8% to 53.8%. This can happen due to the frog images being the most sensitive class to colour, because of most the information that can help identify the frog coming from the colour of the pixels, for example, having a green body.

<hr style="border:2px solid black"> </hr>